In [ ]:
#install cuda-docker -- necessary for many of the benchmarks
%cd /home/chris
!git clone https://github.com/mlperf/reference.git
%cd /home/chris/reference
!bash install_cuda_docker.sh

In [ ]:
#translation
%cd /home/chris/reference/translation
!sudo -H -u chris sh download_data.sh
#!DO WGET COMMANDS
%cd tensorflow
!IMAGE=`sudo docker build . | tail -n 1 | awk '{print $3}'`;SEED=1;NOW=`date "+%F-%T"`;sudo docker run -v $HOME/reference/translation/raw_data:/raw_data --runtime=nvidia -t -i $IMAGE "./run_and_time.sh" $SEED | tee benchmark-$NOW.log

In [ ]:
#recommendation
%cd /home/chris/reference/
!sudo docker pull mlperf/recommendation:v0.5
%cd /home/chris/reference/recommendation/pytorch
#following line only needed if you previously ran the benchmark and want to run the benchmark again from scratch
!rm ml-20m/*
!bash ../download_dataset.sh
!bash ../verify_dataset.sh
!pip install numpy pandas scipy torch
!pip3 install numpy pandas scipy torch
#set seed to an integer between 1 and 5, default here is 1
!docker build -t mlperf/recommendation:v0.5;nvidia-docker run -i -t --rm --ipc=host --mount "type=bind,source=$(pwd),destination=/mlperf/experiment" mlperf/recommendation:v0.5 1 

In [ ]:
#sentiment analysis
%cd /home/chris/reference/sentiment_analysis/paddle
#pull the version of paddle that your system can use
!docker pull paddlepaddle/paddle:latest-gpu-cuda9.0-cudnn7
#change SEED to an integer 1-5
!nvidia-docker run -it -v $PWD:/sentiment_analysis paddlepaddle/paddle:latest-gpu-cuda9.0-cudnn7 /bin/bash -c 'cd sentiment_analysis; sh run_and_time.sh 1'

In [ ]:
#reinforcement
%cd /home/chris/reference/reinforcement/tensorflow/
!IMAGE=`docker build . | tail -n 1 | awk '{print $3}'`;SEED=1;NOW=`date "+%F-%T"`;docker run --runtime=nvidia -t -i $IMAGE "./run_and_time.sh" $SEED | tee benchmark-$NOW.log



In [ ]:
#object detection -- not working
%cd /home/chris
!git clone https://github.com/pytorch/pytorch.git
!cd pytorch; git submodule update --init --recursive
!mv pytorch caffe2
!rm -r reference/object_detection/caffe2/caffe2
!mv caffe2 reference/object_detection/caffe2/
%cd /home/chris/reference/object_detection/caffe2
!git submodule update --init --recursive
!bash ../download_dataset.sh
!bash ../download_weights.sh
!bash extract_dataset.sh
!docker build -t detectron .;nvidia-docker run -v /home/chris/reference/object_detection/caffe2/detectron/lib/datasets/disks/data/coco/:/packages/detectron/lib/datasets/data/coco -it detectron /bin/bash;time stdbuf -o 0 python tools/train_net.py --cfg configs/12_2017_baselines/e2e_mask_rcnn_R-50-FPN_1x.yaml --box_min_ap 0.377 --mask_min_ap 0.339 --seed 3 | tee run.log